# Libraries

In [27]:
# standard
import pandas as pd
import numpy as np
from tqdm import tqdm
from datetime import datetime, date
from collections import defaultdict
import json

# finance
"""
# uncomment for first time installation
!git clone https://github.com/joqueka/bf4py.git
import sys
sys.path.append("bf4py")
"""
import bf4py

# efficiency
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import time

# Data Import

### get list of all German listed companies

https://topforeignstocks.com/stock-lists/the-list-of-listed-companies-in-germany/

In [8]:
traded_companies_path = '../data/Complete-List-of-Listed-Companies-on-Germany-Frankfurt-Stock-Exchange-Jan-2021.xlsx'

traded_companies = pd.DataFrame()

# List the sheet names you want to read
sheet_names_to_read = [1, 2, 3]  # 0-based index, or you can use sheet names as strings

# Loop through each sheet to read and combine data
for sheet_name in sheet_names_to_read:
    # Read the sheet into a DataFrame, skipping the first 7 rows
    df = pd.read_excel(traded_companies_path, sheet_name=sheet_name, skiprows=range(0, 7), usecols=range(0,6))
    
    # Concatenate the newly read DataFrame to the combined DataFrame
    traded_companies = pd.concat([traded_companies, df], ignore_index=True, axis=0)

# only keep german companies
traded_companies = traded_companies[traded_companies['Country'] == 'Germany']
traded_companies.head()

# list for next step
traded_companies_list = traded_companies['ISIN']

del df, sheet_name, sheet_names_to_read, traded_companies_path

### get historical data

In [9]:
# Initialize a defaultdict to store the results
final_dict = defaultdict(dict)

# Create an instance of the General class
general_instance = bf4py.general.General()

# Initialize a list to store ISINs that are not found
not_found_isins = []

# Loop through the list of ISINs
for ISIN in tqdm(traded_companies_list, desc='Getting Share Prices'):
    try:
        history = general_instance.eod_data(
                        isin=ISIN,
                        min_date=datetime.strptime('2010-01-01', '%Y-%m-%d').date(),
                        max_date=date.today()
                        )
        
        # Populate the dictionary
        for record in history:
            final_dict[record['date']][ISIN] = record['close']
    except Exception as e:
        not_found_isins.append(ISIN)

# Convert the dictionary to a DataFrame
final_df = pd.DataFrame.from_dict(final_dict, orient='index')
final_df.index = pd.to_datetime(final_df.index)
final_df.sort_index(inplace=True)

del final_dict, general_instance, history

Getting Share Prices: 100%|██████████| 449/449 [06:36<00:00,  1.13it/s]


### get market cap

In [38]:
# Initialize a dictionary to store the quarterly market cap and number of shares
quarterly_market_cap_dict = {}

# Create an instance of the Equities class
equity_instance = bf4py.equities.Equities()

# Initialize a list to store problematic ISINs
problematic_isins = []

# Loop through each column (ISIN) in the final_df
for ISIN in tqdm(traded_companies_list[:50], desc='Calculating quarterly Market Cap'):
    try:
        # Validate ISIN format (basic validation)
        if len(ISIN) != 12:
            print(f"Skipping invalid ISIN format: {ISIN}")
            continue

        # Resample to get the average share price per quarter
        avg_price_per_quarter = final_df[ISIN].resample('Q').mean()
        
        # Check for NaN
        if pd.isna(avg_price_per_quarter).all():
            avg_price_per_quarter = 1

        # Get the number of shares using the key_data method
        num_shares = equity_instance.key_data(isin=ISIN).get('numberOfShares', 1)  # Default to 1 if not found
        
        # Calculate the quarterly market cap
        quarterly_market_cap = avg_price_per_quarter * num_shares

        # Check for problematic calculations
        if quarterly_market_cap.equals(num_shares):
            quarterly_market_cap = 1

        # Convert dates to 'QX_YYYY' format
        quarters = avg_price_per_quarter.index.to_period("Q").strftime('Q%q_%Y')

        # Store in the dictionary
        quarterly_market_cap_dict[ISIN] = {
            'quarterly_market_cap': quarterly_market_cap.tolist(),
            'quarters': quarters.tolist()
        }
        time.sleep(1)
    except json.JSONDecodeError:
        problematic_isins.append(ISIN)
        time.sleep(1)
    except Exception as e:
        problematic_isins.append(ISIN)
        time.sleep(1)

# Output problematic ISINs
print("Number of Problematic ISINs:", len(problematic_isins))

# Initialize an empty DataFrame to store the final result
market_cap_df = pd.DataFrame()

# Loop through each ISIN and its data in the dictionary
for ISIN, data in quarterly_market_cap_dict.items():
    # Create a temporary DataFrame
    temp_df = pd.DataFrame({
        'Quarter': data['quarters'],
        'Quarterly_Market_Cap': data['quarterly_market_cap']
    })
    # Set the 'Quarter' as the index
    temp_df.set_index('Quarter', inplace=True)
    # Rename the 'Quarterly_Market_Cap' column to the ISIN
    temp_df.rename(columns={'Quarterly_Market_Cap': ISIN}, inplace=True)
    # Merge this temporary DataFrame into the final DataFrame
    if market_cap_df.empty:
        market_cap_df = temp_df
    else:
        market_cap_df = pd.merge(market_cap_df, temp_df, left_index=True, right_index=True, how='outer')

Calculating quarterly Market Cap: 100%|██████████| 449/449 [09:33<00:00,  1.28s/it]

Number of Problematic ISINs: 118


In [45]:
# Create a mapping dictionary from the company_info_df
isin_to_symbol = dict(zip(traded_companies['ISIN'], traded_companies['Trading Symbol']))

# Rename the columns in the final_df
final_df.rename(columns=isin_to_symbol, inplace=True)
market_cap_df.rename(columns=isin_to_symbol, inplace=True)

# save file
final_df.to_csv('../data/closing_prices.csv')
market_cap_df.to_csv('../data/market_cap.csv')